In [43]:
from openpyxl import load_workbook
from openpyxl import workbook
import os
import win32com.client as win32
import pandas as pd
from unidecode import unidecode

In [44]:
## COVERTER DE XLS PARA XLSX
open_filepath = r'C:\Users\vinicius.perazza\Desktop\PROJETOS\ESTUDO\industria\xls\dados_industria.xls'
save_filepath = r'C:\Users\vinicius.perazza\Desktop\PROJETOS\ESTUDO\industria\xlsx\dados_industria.xls'

xlsx_format = save_filepath+"x"

#Verifica se já existe um arquivo e remove
if os.path.exists(xlsx_format) == True:
    os.remove(xlsx_format)
    
excel = win32.Dispatch('Excel.Application')
#excel = win32.gencache.EnsureDispatch('Excel.Application')
wb = excel.Workbooks.Open(open_filepath)

wb.SaveAs(save_filepath+"x", FileFormat = 51)    #FileFormat = 51 is for .xlsx extension
wb.Close()                                       #FileFormat = 56 is for .xls
excel.Application.Quit()

In [45]:
##CARREGANDO PLANILHA
wb = load_workbook(xlsx_format)
sheets = wb.sheetnames
print(sheets)

#SELECIONAR UMA WORKSHEETS (inicia em 0)
sheets = sheets[3] # <- de 0 até 15

ws = wb[sheets]
print(ws, "Ativo")

['PRODUÇÃO', 'EMPREGADOS', 'UCI (%)', 'UCI (efetiva-usual)', 'ESTOQUES (evolução)', 'ESTOQUES (efetivo-planejado)', 'EXPECTATIVAS - DEMANDA', 'EXPECTATIVA - EXPORTAÇÃO', 'EXPECTATIVA - COMPRAS', 'EXPECTATIVA - EMPREGADOS', 'EXPECTATIVA - INVESTIMENTO', 'SITUACAO FINANCEIRA LUCRO', 'SITUACAO FINANCEIRA PREÇO MEDIO', 'SITUACAO FINANCEIRA', 'SITUACAO FINANCEIRA CREDITO', 'PRINCIPAIS_PROBLEMAS']
<Worksheet "UCI (efetiva-usual)"> Ativo


# COLETA DE DADOS

In [46]:
##LENDO OS MESES
    
max_linha = ws.max_row
max_coluna = ws.max_column

valores = []

for i in range(2, max_coluna + 1):
    valores.append(ws.cell(row=8, column=i).value)

df_dados = pd.DataFrame(valores,  columns =  ["Data"])
    
##CRIANDO DATA FRAME COM OS DADOS DE PRODUÇÃO

linhas = 11

while linhas <= 54:
    
    if linhas > 54:
        continue
        
    max_coluna = ws.max_column
    valores_r = []
    
    for i in range(linhas, linhas + 1):
        var = (ws.cell(row=i, column = 1).value)
    for j in range(2, max_coluna + 1): 
        valores_r.append(ws.cell(row = i, column=j).value)
    df_dados.insert(1, var, valores_r)
    
    linhas += 1

# LIMPANDO OS DADOS

In [47]:
#RETIRA AS COLUNAS VAZIAS
df_dados = df_dados.drop(columns = ["POR PORTE", "POR SETOR"])

#RETIRA LINHA VAZIAS
df_dados.dropna()

,Data,"Manutenção, reparação e instalação de máquinas e equipamentos.",Produtos diversos,Móveis,Outros equipamentos de transporte,"Veículos automotores, reboques e carrocerias",Máquinas e equipamentos,"Máquinas, aparelhos e materiais elétricos","Equipamentos de informática, produtos eletrônicos e outros",Produtos de metal (exceto máquinas e equipamentos),...,"Extração de carvão, petróleo e gás natural",IND. EXTRATIVA,GRANDE,MÉDIA,PEQUENA,CENTRO-OESTE,SUL,SUDESTE,NORDESTE,NORTE
0,jun09,-,40.8,34.8,-,30.9,32.8,30.0,35.8,28.9,...,-,37.6,37.1,36.7,34.8,40.2,32.2,36.3,41.0,37.0
1,set09,-,45.2,43.5,-,42.4,38.3,42.2,42.0,41.9,...,-,40.6,46.6,43.4,43.5,45.6,42.0,44.8,49.6,45.8
2,dez09,-,53.9,45.3,-,44.0,44.2,49.0,52.1,47.0,...,-,43.7,49.5,48.0,45.8,46.6,46.7,48.6,50.6,46.8
3,jan10,-,48.1,43.1,-,46.9,52.7,46.6,48.8,45.4,...,-,43.1,50.5,46.7,45.5,50.7,48.6,46.8,48.2,53.4
4,fev10,-,47.3,46.6,-,45.3,51.1,47.1,56.3,49.3,...,-,46.8,51.6,46.2,45.1,48.6,51.9,46.7,48.6,44.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,fev20,45.5,46.1,42.5,47.1,40.1,41.9,41.9,41.2,42.9,...,-,46.0,47.4,42.3,41.3,45.6,47.9,43.4,43.6,41.9
125,mar20,33,28.9,15.8,23.2,27.1,26.7,33.3,27.1,25.2,...,-,45.2,32.8,31.8,26.8,31.6,29.1,29.8,28.7,33.2
126,abr20,27.8,19.6,8.6,17.2,13.9,22.3,22.0,17.6,19.0,...,-,40.5,24.3,24.6,22.3,34.9,19.7,24.1,22.9,24.6
127,mai20,28.4,23.4,20.9,25,27.1,29.8,30.6,19.7,27.0,...,-,42.0,30.5,29.6,28.0,36.7,28.4,29.2,27.9,28.8


# CORRIGINDO O CAMPO DATA

In [48]:
df_data = df_dados["Data"]

#TRANSFORMA A DATA PARA PADRAO EN
df_data = df_data.replace({'fev': 'feb'}, regex=True)
df_data = df_data.replace({'abr': 'apr'}, regex=True)
df_data = df_data.replace({'mai': 'may'}, regex=True)
df_data = df_data.replace({'ago': 'aug'}, regex=True)
df_data = df_data.replace({'set': 'sep'}, regex=True)
df_data = df_data.replace({'out': 'oct'}, regex=True)
df_data = df_data.replace({'dez': 'dec'}, regex=True)

#FORMATA A DATA COM FORMATO MES/ANO PARA DATETIME
df_data = pd.to_datetime(df_data, format='%b%y', errors="coerce").fillna(pd.to_datetime(df_data, format="%Y-%m-%d %H:%M:%S", errors="coerce"))

df_data = pd.DataFrame(df_data)

In [49]:
df_data.head(76)

,Data
0,2009-06-01
1,2009-09-01
2,2009-12-01
3,2010-01-01
4,2010-02-01
...,...
71,2015-09-01
72,2015-10-01
73,2015-11-01
74,2015-12-01


In [50]:
#DROPA A COLUNA DATA DO DATAFRAME COM OS DADOS
df_dados = df_dados.drop(columns="Data")

In [51]:
#MERGE NA SERIE COM AS DATAS E O DATAFRAME DOS DADOS
df_dados = pd.merge(df_dados, df_data, left_index=True, right_index=True)
df_dados

,"Manutenção, reparação e instalação de máquinas e equipamentos.",Produtos diversos,Móveis,Outros equipamentos de transporte,"Veículos automotores, reboques e carrocerias",Máquinas e equipamentos,"Máquinas, aparelhos e materiais elétricos","Equipamentos de informática, produtos eletrônicos e outros",Produtos de metal (exceto máquinas e equipamentos),Metalurgia,...,IND. EXTRATIVA,GRANDE,MÉDIA,PEQUENA,CENTRO-OESTE,SUL,SUDESTE,NORDESTE,NORTE,Data
0,-,40.8,34.8,-,30.9,32.8,30.0,35.8,28.9,25.8,...,37.6,37.1,36.7,34.8,40.2,32.2,36.3,41.0,37.0,2009-06-01
1,-,45.2,43.5,-,42.4,38.3,42.2,42.0,41.9,35.4,...,40.6,46.6,43.4,43.5,45.6,42.0,44.8,49.6,45.8,2009-09-01
2,-,53.9,45.3,-,44.0,44.2,49.0,52.1,47.0,40.9,...,43.7,49.5,48.0,45.8,46.6,46.7,48.6,50.6,46.8,2009-12-01
3,-,48.1,43.1,-,46.9,52.7,46.6,48.8,45.4,44.4,...,43.1,50.5,46.7,45.5,50.7,48.6,46.8,48.2,53.4,2010-01-01
4,-,47.3,46.6,-,45.3,51.1,47.1,56.3,49.3,47.6,...,46.8,51.6,46.2,45.1,48.6,51.9,46.7,48.6,44.2,2010-02-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
131,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
132,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
133,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [52]:
df_dados = df_dados.dropna()

In [53]:
df_dados.columns[2]

'Móveis'

In [54]:
#RENOMEIA AS COLUNAS COLOCANDO NO PADRAO
df_dados = df_dados.rename(columns={'Manutenção, reparação e instalação de máquinas e equipamentos.': 'manutencao reparacao e instalacao de maquinas e equipamentos'
                                ,'Produtos diversos': 'produtos diversos'
                                ,'Móveis': 'moveis'
                                ,'Outros equipamentos de transporte': 'outros equipamentos de transporte'
                                ,'Veículos automotores, reboques e carrocerias ': 'veiculos automotores reboques e carrocerias'
                                ,'Máquinas e equipamentos': 'maquinas e equipamentos'
                                ,'Máquinas, aparelhos e materiais elétricos ': 'maquinas aparelhos e materiais eletricos'
                                ,'Equipamentos de informática, produtos eletrônicos e outros ': 'equipamentos de informatica produtos eletronicos e outros'
                                ,'Produtos de metal (exceto máquinas e equipamentos)  ': 'produtos de metal'
                                ,'Metalurgia': 'metalurgia'
                                ,'Produtos de minerais não metálicos': 'produtos de minerais nao metalicos'
                                ,'Produtos de material plástico': 'produtos de material plastico'
                                ,'Produtos de borracha': 'produtos de borracha'
                                ,'Produtos farmoquímicos e farmacêuticos ': 'produtos farmoquimicos e farmaceuticos'
                                ,'Sabões, detergentes, produtos de limpeza, cosméticos, produtos de perfumaria e de higiene pessoal (HPPC)': 'hppc'
                                ,'Químicos (exceto HPPC)': 'quimicos'
                                ,'Biocombustíveis': 'biocombustiveis'
                                ,'Coque e derivados do petróleo': 'coque e derivados do petroleo'
                                ,'Impressão e reprodução de gravações': 'impressao e reproducao de gravacoes'
                                ,'Celulose, papel e produtos de papel': 'celulose papel e produtos de papel'
                                ,'Produtos de madeira': 'produtos de madeira'
                                ,'Calçados e suas partes': 'calcados e suas partes'
                                ,'Couros e artefatos de couro': 'couros e artefatos de couro'
                                ,'Confecção de artigos do vestuário e acessórios ': 'confeccao de artigos do vestuario e acessorios'
                                ,'Produtos têxteis': 'produtos texteis'
                                ,'Produtos do fumo': 'produtos do fumo'
                                ,'Bebidas': 'bebidas'
                                ,'Produtos alimentícios': 'produtos alimenticios'
                                ,'IND. DE TRANSFORMAÇÃO': 'industria de transformacao'
                                ,'Atividades de apoio à extração de minerais': 'atividades de apoio a extracao de minerais'
                                ,'Extração de minerais não metálicos': 'extracao de minerais nao metalicos'
                                ,'Extração de minerais metálicos': 'extracao de minerais metalicos'
                                ,'Extração de carvão, petróleo e gás natural': 'extracao de carvao petroleo e gas natural'
                                ,'IND. EXTRATIVA': 'industria extrativa'
                                ,'GRANDE': 'industria grande'
                                ,'MÉDIA': 'industria media'
                                ,'PEQUENA ': 'industria pequena'
                                ,'CENTRO-OESTE': 'regiao centro oeste'
                                ,'SUL': 'regiao sul'
                                ,'SUDESTE': 'regiao sudeste'
                                ,'NORDESTE': 'regiao nordeste'
                                ,'NORTE': 'regiao norte'
                                ,'Data': 'data'
                            })
df_dados.head()

,manutencao reparacao e instalacao de maquinas e equipamentos,produtos diversos,moveis,outros equipamentos de transporte,veiculos automotores reboques e carrocerias,maquinas e equipamentos,maquinas aparelhos e materiais eletricos,equipamentos de informatica produtos eletronicos e outros,produtos de metal,metalurgia,...,industria extrativa,industria grande,industria media,industria pequena,regiao centro oeste,regiao sul,regiao sudeste,regiao nordeste,regiao norte,data
0,-,40.8,34.8,-,30.9,32.8,30.0,35.8,28.9,25.8,...,37.6,37.1,36.7,34.8,40.2,32.2,36.3,41.0,37.0,2009-06-01
1,-,45.2,43.5,-,42.4,38.3,42.2,42.0,41.9,35.4,...,40.6,46.6,43.4,43.5,45.6,42.0,44.8,49.6,45.8,2009-09-01
2,-,53.9,45.3,-,44.0,44.2,49.0,52.1,47.0,40.9,...,43.7,49.5,48.0,45.8,46.6,46.7,48.6,50.6,46.8,2009-12-01
3,-,48.1,43.1,-,46.9,52.7,46.6,48.8,45.4,44.4,...,43.1,50.5,46.7,45.5,50.7,48.6,46.8,48.2,53.4,2010-01-01
4,-,47.3,46.6,-,45.3,51.1,47.1,56.3,49.3,47.6,...,46.8,51.6,46.2,45.1,48.6,51.9,46.7,48.6,44.2,2010-02-01


In [55]:
#TRANSFORMANDO OS VALORES "-" EM "0"
#ALTERANDO "." PARA ","

df_dados = df_dados.replace('-','0')
df_dados = df_dados.replace('.',',')
df_dados

,manutencao reparacao e instalacao de maquinas e equipamentos,produtos diversos,moveis,outros equipamentos de transporte,veiculos automotores reboques e carrocerias,maquinas e equipamentos,maquinas aparelhos e materiais eletricos,equipamentos de informatica produtos eletronicos e outros,produtos de metal,metalurgia,...,industria extrativa,industria grande,industria media,industria pequena,regiao centro oeste,regiao sul,regiao sudeste,regiao nordeste,regiao norte,data
0,0,40.8,34.8,0,30.9,32.8,30.0,35.8,28.9,25.8,...,37.6,37.1,36.7,34.8,40.2,32.2,36.3,41.0,37.0,2009-06-01
1,0,45.2,43.5,0,42.4,38.3,42.2,42.0,41.9,35.4,...,40.6,46.6,43.4,43.5,45.6,42.0,44.8,49.6,45.8,2009-09-01
2,0,53.9,45.3,0,44.0,44.2,49.0,52.1,47.0,40.9,...,43.7,49.5,48.0,45.8,46.6,46.7,48.6,50.6,46.8,2009-12-01
3,0,48.1,43.1,0,46.9,52.7,46.6,48.8,45.4,44.4,...,43.1,50.5,46.7,45.5,50.7,48.6,46.8,48.2,53.4,2010-01-01
4,0,47.3,46.6,0,45.3,51.1,47.1,56.3,49.3,47.6,...,46.8,51.6,46.2,45.1,48.6,51.9,46.7,48.6,44.2,2010-02-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,45.5,46.1,42.5,47.1,40.1,41.9,41.9,41.2,42.9,45.3,...,46.0,47.4,42.3,41.3,45.6,47.9,43.4,43.6,41.9,2020-02-01
125,33,28.9,15.8,23.2,27.1,26.7,33.3,27.1,25.2,30.3,...,45.2,32.8,31.8,26.8,31.6,29.1,29.8,28.7,33.2,2020-03-01
126,27.8,19.6,8.6,17.2,13.9,22.3,22.0,17.6,19.0,23.3,...,40.5,24.3,24.6,22.3,34.9,19.7,24.1,22.9,24.6,2020-04-01
127,28.4,23.4,20.9,25,27.1,29.8,30.6,19.7,27.0,28.6,...,42.0,30.5,29.6,28.0,36.7,28.4,29.2,27.9,28.8,2020-05-01


In [56]:
#SALVANDO DATAFRAME EM CSV

save_filepath = r'C:\Users\vinicius.perazza\Desktop\PROJETOS\ESTUDO\industria\csv\dados_industria'+ '_' + sheets + '.csv'

df_dados.to_csv(save_filepath, decimal = ",", sep = ";")